In [1]:
# import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import substring, size

spark = SparkSession.builder.appName('my app').master('local').getOrCreate()
sc = spark.sparkContext

dataschema = StructType([
   StructField("playerID", StringType(), False),
   StructField("awardID", StringType(), False),
   StructField("yearID", LongType(), False),
   StructField("lgID", StringType(), False),
   StructField("tie", LongType(), False),
   StructField("notes", LongType(), False)
 ])

salaryschema = StructType([
   StructField("yearID", LongType(), False),
   StructField("teamID", StringType(), False),
   StructField("lgID", StringType(), False),
   StructField("playerID", StringType(), False),
   StructField("salary", LongType(), False)
 ])

award_df = spark.read.format("csv").option("sep", ",").option('header', True).schema(dataschema).load("./input_week06/AwardsPlayers.csv")
salary_df = spark.read.format("csv").option("sep", ",").option('header', True).schema(salaryschema).load("./input_week06/Salaries.csv")


22/10/17 13:39:31 WARN Utils: Your hostname, bagdoyeong-ui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.0.13 instead (on interface en0)
22/10/17 13:39:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/17 13:39:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf, expr

문제 1.

  3년 연속 같은 상을 수상한 선수를 찾아 그 3년간의 연봉평균을 출력하시오
  만일, 3년 초과 같은 상을 수상하였다면 모든 연속된 3년에 대해 연봉평균을 출력하시오

In [3]:
award_df.sort('playerID', 'awardID', 'yearID').show()

+---------+--------------------+------+----+----+-----+
| playerID|             awardID|yearID|lgID| tie|notes|
+---------+--------------------+------+----+----+-----+
| Brecheen|Baseball Magazine...|  1945|  ML|null| null|
| Brecheen|Baseball Magazine...|  1945|  NL|null| null|
| Brecheen|Baseball Magazine...|  1946|  NL|null| null|
| Brecheen|Baseball Magazine...|  1948|  NL|null| null|
|aaronha01|          Gold Glove|  1958|  NL|null| null|
|aaronha01|          Gold Glove|  1959|  NL|null| null|
|aaronha01|          Gold Glove|  1960|  NL|null| null|
|aaronha01|Lou Gehrig Memori...|  1970|  ML|null| null|
|aaronha01|Most Valuable Player|  1957|  NL|null| null|
|aaronha01|        TSN All-Star|  1956|  ML|null| null|
|aaronha01|        TSN All-Star|  1958|  ML|null| null|
|aaronha01|        TSN All-Star|  1959|  ML|null| null|
|aaronha01|        TSN All-Star|  1963|  NL|null| null|
|aaronha01|        TSN All-Star|  1965|  NL|null| null|
|aaronha01|        TSN All-Star|  1967|  NL|null

In [4]:
award_df_cnt = award_df.groupBy('playerID').count()
award_df_filter = award_df_cnt.filter(award_df_cnt['count']>=3)
award_df_filter = award_df_filter.selectExpr('playerID as playID')
award_ = award_df.join(award_df_filter, award_df['playerID']==award_df_filter['playID'], 'inner')

In [5]:
award_ = award_.sort('playerID', 'awardID', 'yearID').select('playerID', 'awardID', 'yearID', 'lgID')
award_group = award_.groupBy('playerID', 'awardID').agg(F.collect_set('yearID').alias('year'))

In [6]:
award_group_cnt = award_group.select('*', size('year').alias('cnt'))
award_group_cnt = award_group_cnt.filter(award_group_cnt['cnt']>=3)
award_group_cnt.show()

+---------+--------------------+--------------------+---+
| playerID|             awardID|                year|cnt|
+---------+--------------------+--------------------+---+
| Brecheen|Baseball Magazine...|  [1948, 1945, 1946]|  3|
|aaronha01|          Gold Glove|  [1958, 1959, 1960]|  3|
|aaronha01|        TSN All-Star|[1965, 1969, 1958...|  9|
|alexape01|Baseball Magazine...|[1917, 1914, 1915...| 10|
|alexape01|Pitching Triple C...|  [1915, 1920, 1916]|  3|
|allenjo02|Baseball Magazine...|  [1936, 1937, 1938]|  3|
|alomaro01|          Gold Glove|[2001, 1998, 1991...| 10|
|alomaro01|      Silver Slugger|[1992, 1999, 1996...|  4|
|alomaro01|        TSN All-Star|[1998, 1992, 1999...|  5|
|aparilu01|          Gold Glove|[1968, 1966, 1958...|  9|
|aparilu01|        TSN All-Star|[1968, 1972, 1966...|  5|
|applilu01|Baseball Magazine...|[1936, 1940, 1937...|  4|
|applilu01|        TSN All-Star|  [1936, 1940, 1943]|  3|
|archeji01|Baseball Magazine...|[1914, 1911, 1912...|  5|
|ausmubr01|   

In [7]:
def check_is_consecutive(l):
     l = sorted(l)
     result = []
     for i in range(len(l)-2):
          if l[i] + 2 == l[i+1] + 1 == l[i+2]:
               result.append([l[i], l[i+1], l[i+2]])

     return result

In [8]:
check_is_consecutive([1,2,3,5,6,7])

[[1, 2, 3], [5, 6, 7]]

In [9]:
udf_sort = udf(lambda x: check_is_consecutive(x))
award_bool = award_group_cnt.select('*', udf_sort(award_group_cnt['year']).alias('type'))
award_bool.show()

+---------+--------------------+--------------------+---+--------------------+
| playerID|             awardID|                year|cnt|                type|
+---------+--------------------+--------------------+---+--------------------+
| Brecheen|Baseball Magazine...|  [1948, 1945, 1946]|  3|                  []|
|aaronha01|          Gold Glove|  [1958, 1959, 1960]|  3|[[1958, 1959, 1960]]|
|aaronha01|        TSN All-Star|[1965, 1969, 1958...|  9|[[1969, 1970, 1971]]|
|alexape01|Baseball Magazine...|[1917, 1914, 1915...| 10|[[1913, 1914, 191...|
|alexape01|Pitching Triple C...|  [1915, 1920, 1916]|  3|                  []|
|allenjo02|Baseball Magazine...|  [1936, 1937, 1938]|  3|[[1936, 1937, 1938]]|
|alomaro01|          Gold Glove|[2001, 1998, 1991...| 10|[[1991, 1992, 199...|
|alomaro01|      Silver Slugger|[1992, 1999, 1996...|  4|                  []|
|alomaro01|        TSN All-Star|[1998, 1992, 1999...|  5|[[1998, 1999, 2000]]|
|aparilu01|          Gold Glove|[1968, 1966, 1958...

문제 2.

NL리그에서 가장 많이 이긴 팀을 찾으시오

In [10]:
win_df = spark.read.format("csv").option("sep", ",").option('header', True).load("./input_week06/Teams.csv")

In [11]:
win_df.select('lgID', 'teamID', 'W').show()

+----+------+---+
|lgID|teamID|  W|
+----+------+---+
|  NA|   BS1| 20|
|  NA|   CH1| 19|
|  NA|   CL1| 10|
|  NA|   FW1|  7|
|  NA|   NY2| 16|
|  NA|   PH1| 21|
|  NA|   RC1|  4|
|  NA|   TRO| 13|
|  NA|   WS3| 15|
|  NA|   BL1| 35|
|  NA|   BR1|  3|
|  NA|   BR2|  9|
|  NA|   BS1| 39|
|  NA|   CL1|  6|
|  NA|   MID|  5|
|  NA|   NY2| 34|
|  NA|   PH1| 30|
|  NA|   TRO| 15|
|  NA|   WS3|  2|
|  NA|   WS4|  0|
+----+------+---+
only showing top 20 rows



In [12]:
win_ = win_df.withColumn('W', win_df['W'].cast('long')).select('lgID', 'teamID', 'W')

In [13]:
win_sum = win_.groupBy('teamID').sum('W').withColumnRenamed('sum(W)', 'sum_W')

In [14]:
win_sum.sort('sum_W', ascending=False).show(1)

+------+-----+
|teamID|sum_W|
+------+-----+
|   CHN|10372|
+------+-----+
only showing top 1 row

